In [2]:
import numpy as np
import pickle
import crypto

def productModp(prodList, p):
    prod = 1
    for t in prodList:
        prod = (prod * t) % p
    return prod
# Read needed private and public data
with open('privateInfo','rb') as fi:
    info = pickle.load(fi)
with open('BB','rb') as fbb:
    Pub = pickle.load(fbb)
ck, sk, s, n, m, t, r, C, perm = info["ck"], info["sk"], info["s"], info["n"], info["m"], info["t"], info["r"], info["C"], info["perm"]
x = max(n+1, 2*m-1)
p,q,g,h = ck
Publ = Pub["Publ"]
U = np.array([pl[1] for pl in Publ])
E = np.array([pl[2] for pl in Publ])
# Commit function
Com = lambda ck, M, r: crypto.encrypt(M, ck, r)
Dec = lambda sk, C, r: crypto.decrypt(C, sk, r)
### tally ###
with open('votes','rb') as fv:
    votes = pickle.load(fv)

# Create tuples (vote, audit side of double-ballot) for each voter and commit to BB
auditVS = []
for v in votes:
    auditVS.append((s[v[0] - 1][2-v[1]], v))
# Open vote_code committments
Uopen = [[[(Dec(sk, U[i,a,j], t[i,a,j]), t[i,a,j]) for j in range(m)] for a in range(2)]for i in range(n)]
# Open rank committments from the side not voted
Eopen = [E[v[0]-1,1-v[1]] for v in votes]
ropen = [r[v[0]-1, 1-v[1]] for v in votes]
### Change how this is done?
Etally = [[] for i in range(m)]
rand = [[] for i in range(m)]
for v in votes:
    # Ord[vote_code] == corresponding rank
    Ord = dict(zip(C[v[0]-1][v[1]], range(m)))
    for j in range(m):
        temp = perm[v[0]-1, v[1], Ord[int(v[2][j])]]
        Etally[j].append(E[v[0]-1, v[1], temp])
        rand[j].append(r[v[0]-1, v[1], temp])

Esum = [productModp(tally, p) for tally in Etally]
R = [sum(randList) for randList in rand]
T = [Dec(sk, Esum[i], R[i]) for i in range(m)]
Open = [[(Dec(sk, Eopen[i][j], ropen[i][j]), ropen[i][j]) for j in range(m)] for i in range(n)]

bb = {
    "Pub": Pub,
    "OpenBallots": auditVS,
    "Uopen": Uopen,
    "Result": (Etally, Esum, (T,R)),
    "Eopen": (Open, Eopen)
}

with open('BB', 'wb') as fbb:
    pickle.dump(bb, fbb)

In [3]:
### result ###
# given decommitted value of result (polynomial of x of order m) compute res = [s_0,s_1,...,s_{m-1}] where
# s_i is the number of times that this candidate was given a rank of i.
def produceResult(decommitment):
    res = []
    T = decommitment
    for j in range(m):
        s = T % x
        res.append(s)
        T = (T - s) // x
    return res
    
fullRes = [produceResult(t) for t in T]
res = []
for x in fullRes:
    result = 0
    for i in range(m):
        result += i*x[i]
    res.append(result)
print(fullRes)
print(res)

[[2, 1, 3, 4], [2, 2, 3, 3], [2, 4, 2, 2], [4, 3, 2, 1]]
[19, 17, 14, 10]
